In [1]:
import os
import math
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score

# from scripts.preprocessing import preprocess_data


In [2]:
os.chdir("../")

In [ ]:
train_data = pd.read_csv("data/rossmann/train.csv")
store_data = pd.read_csv("data/rossmann/store.csv")

In [4]:
df = pd.merge(train_data, store_data, on='Store')


In [8]:
df = df.head(100000)

In [ ]:
from scripts.preprocessing import preprocess_data


df, label_encoders, scaler = preprocess_data(df)



In [19]:
# Handle infinite and NaN values in X
X = df.drop(columns=['Sales'])  # Drop the 'Sales' column to use the remaining as features
X = X.replace([np.inf, -np.inf], np.nan)  # Replace infinity values with NaN
X = X.fillna(X.mode())  # Fill NaN values with the mode of the column

y = df['Sales']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define numerical and categorical columns
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X_train.select_dtypes(include=['object']).columns  # Identify string columns

# Define preprocessor with ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numerical_cols),  # Scale numerical columns
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)  # OneHotEncode categorical columns
])

# Build a pipeline with preprocessor and RandomForestRegressor
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Preprocess both numerical and categorical features
    ('regressor', RandomForestRegressor(random_state=42))  # RandomForest model
])

# Train the model
pipeline.fit(X_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

# Output the results

print("Root Mean Squared Error (RMSE):", rmse)
print(f'Mean Squared Error (MSE): {mse}')
print(f'R² Score: {r2}')


Mean Squared Error (MSE): 296313.38951113995
R² Score: 0.977364892043033
